In [1]:
import tensorflow as tf
tf.config.gpu.set_per_process_memory_fraction(0.3)
tf.config.gpu.set_per_process_memory_growth(True)

## Methods

In [3]:
##image loader

import os
import numpy as np
from sklearn.utils import shuffle
from skimage.io import imread
from skimage.transform import resize
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator

def path_loader(fold1, fold2, data_path):
    #Creating data path
    image_data_path = os.path.join(data_path, fold1)   
    mask_data_path = os.path.join(data_path, fold2)
    images = []
    masks = []
    #Listing all file names in the path

    
    for root, dirs, files in os.walk(image_data_path):
        for name in files:
            images.append(os.path.join(image_data_path,name))
            
    for root2, dirs2, files2 in os.walk(mask_data_path):
        for name2 in files2:
            masks.append(os.path.join(mask_data_path,name2))
    images.sort()
    masks.sort()
            

    return images, masks
    

# reading and resizing the training images with their corresponding labels
def get_train_data_shuffled(images, masks, p):
    

    images, masks = shuffle(images,masks)
    
    #train_x, test_x, train_y, test_y = train_test_split(images,masks,test_size = p)

    return images, masks

def data_loader(fold1, fold2, data_path, p,img_h, img_w):
    
    images, masks = path_loader(fold1, fold2, data_path)
    train_x, train_y = get_train_data_shuffled(images, masks, p)
    
    train_img = []
    train_mask = []
    test_img = []
    test_mask = []
    len(train_x)
    for i in range(len(train_x)):
        image_name = train_x[i]
        img = imread(image_name, as_grey=True)
        img = resize(img, (img_h, img_w), anti_aliasing = False,preserve_range=True, order = 0).astype('float32')
        train_img.append([np.array(img)]) 
        
        mask_name = train_y[i]
        mask = imread(mask_name, as_grey=True)
        mask = resize(mask, (img_h, img_w), anti_aliasing = False,preserve_range=True, order = 0).astype('float32')
        train_mask.append([np.array(mask)])

        if i % 500 == 0:
            print('Reading: {0}/{1}  of train images'.format(i, len(train_x)))
            print("Readin image {} and mask {}".format(image_name,mask_name))
    return train_img, train_mask
"""
    for j in range(len(train_y)):
        mask_name = train_y[j]
        mask = imread(mask_name, as_grey=True)
        mask = resize(mask, (img_h, img_w), anti_aliasing = False,preserve_range=True, order = 0).astype('float32')
        train_mask.append([np.array(mask)])
    """
        
        
 
   

# Instantiating images and labels for the model.
def get_train_test_data(fold1, fold2, data_path, p,img_h, img_w):
    
    train_img, train_mask = data_loader(fold1, fold2, data_path, p,img_h, img_w)

    Train_Img = np.zeros((len(train_img), img_h, img_w), dtype = np.float32)
    Train_Label = np.zeros((len(train_mask),img_h, img_w), dtype = np.int32)
    Weight_Map = np.zeros((len(train_mask),img_h, img_w), dtype = np.int32)
    
    for i in range(len(train_img)):
        Train_Img[i] = train_img[i][0]
        mask_semi = train_mask[i][0]
        mask_semi[mask_semi>0]=1
        Train_Label[i] = mask_semi
        Weight_Map[i] = create_weight_map(Train_Label[i], 2, i)
    
    Train_Img = np.expand_dims(Train_Img, axis = 3)  
    Train_Label = np.expand_dims(Train_Label, axis = 3) 

    
    print(Train_Img.shape)
   
    return Train_Img, Train_Label, Weight_Map

In [2]:
p = 0.2
path = '/Lab1/Lab3/MRI/'
fold1 = 'Image'
fold2 = 'Mask'
image_size = 240

train,test, weight = get_train_test_data(fold1, fold2, path, p,image_size, image_size)



/usr/local/lib/python3.5/dist-packages/skimage/io/_io.py:48: UserWarning: `as_grey` has been deprecated in favor of `as_gray`
  warn('`as_grey` has been deprecated in favor of `as_gray`')


Reading: 0/9403  of train images
Readin image /Lab1/Lab3/MRI/Image/Brats17_CBICA_ASW_1_t1ce_53.png and mask /Lab1/Lab3/MRI/Mask/Brats17_CBICA_ASW_1_t1ce_53_Tumor.png


KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(4,4))
plt.imshow(train[300,:,:,0])
plt.colorbar()

a = plt.figure(figsize=(4,4))
plt.imshow(test[300,:,:,0])
plt.colorbar()
a.savefig('/Plots/Task1/Test.png')

plt.figure(figsize=(4,4))
plt.imshow(weight[300,:,:])
plt.colorbar()

In [4]:
##Data Augmentation
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img

def DataAugmentation(rotation_range,width_shift,height_shift_range,rescale,horizontal_flip):
    
    #Train data
    train_datagen = ImageDataGenerator(rotation_range = rotation_range, width_shift_range = width_shift, height_shift_range=height_shift_range,
                                       horizontal_flip = horizontal_flip, rescale = rescale)
   
    #Val data
    val_datagen = ImageDataGenerator(rescale = rescale)
 
    
    
    return train_datagen, val_datagen 

In [5]:
#Similarity metrics
from tensorflow.keras import backend as K
smooth = 1.

def dice_coef(y_true, y_pred):
    #y_true_f = K.flatten(y_true)
    #y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true * y_pred,axis = [1,2,3])
    return K.mean((2. * intersection + smooth) / (K.sum(y_true, axis = [1,2,3]) + K.sum(y_pred, axis = [1,2,3]) + smooth), axis = 0)

def dice_coef_loss(y_true, y_pred):
    return 1-dice_coef(y_true, y_pred)

In [6]:
##model

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, Input, concatenate, Conv2DTranspose
from tensorflow.keras.layers import Dropout, BatchNormalization, SpatialDropout2D
from tensorflow.keras.layers import MaxPooling2D

def u_net(Base,img_height, img_width, img_ch, batchNormalization, SDRate, spatial_dropout, final_neurons, final_afun):
    inputs = Input((img_height, img_width, img_ch))
    inputs2 = Input((img_height, img_width, img_ch))
    
    ## Contraction
    # Conv Block 1
    
    c1 = Conv2D(filters=Base,
                     kernel_size=(3,3), strides=(1,1), padding='same')(inputs)
    
     #Add batch Normalization
    if batchNormalization:
        c1 = BatchNormalization(axis=-1)(c1)
    
    a1 = Activation('relu')(c1)
    
    #Add spatial Dropout
    if spatial_dropout:
        a1 = SpatialDropout2D(SDRate)(a1)
        
    c2 = Conv2D(filters=Base,
                     kernel_size=(3,3), strides=(1,1), padding='same')(a1)
    
     #Add batch Normalization
    if batchNormalization:
        c2 = BatchNormalization(axis=-1)(c2)
    
    a2 = Activation('relu')(c2)
    
    #Add spatial Dropout
    if spatial_dropout:
        a2 = SpatialDropout2D(SDRate)(a2)
        
    m1 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(a2)
        
    # Conv Block 2
    c3 = Conv2D(filters=Base*2,
                     kernel_size=(3,3), strides=(1,1), padding='same')(m1)
    
     #Add batch Normalization
    if batchNormalization:
        c3 = BatchNormalization(axis=-1)(c3)
    
    a3 = Activation('relu')(c3)
    
    #Add spatial Dropout
    if spatial_dropout:
        a3 = SpatialDropout2D(SDRate)(a3)
    
    c4 = Conv2D(filters=Base*2,
                     kernel_size=(3,3), strides=(1,1), padding='same')(a3)
    
     #Add batch Normalization
    if batchNormalization:
        c4 = BatchNormalization(axis=-1)(c4)
    
    a4 = Activation('relu')(c4)
    
    #Add spatial Dropout
    if spatial_dropout:
        a4 = SpatialDropout2D(SDRate)(a4)
    
    m2 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(a4)
    
    # Conv Block 3
    c5 = Conv2D(filters=Base*4, 
                     kernel_size=(3,3), strides=(1,1), padding='same')(m2)
    
     #Add batch Normalization
    if batchNormalization:
        c5 = BatchNormalization(axis=-1)(c5)
    
    a5 = Activation('relu')(c5)
    
    #Add spatial Dropout
    if spatial_dropout:
        a5 = SpatialDropout2D(SDRate)(a5)
        
    c6 = Conv2D(filters=Base*4,
                     kernel_size=(3,3), strides=(1,1), padding='same')(a5)
    
     #Add batch Normalization
    if batchNormalization:
          c6 = BatchNormalization(axis=-1)(c6)
    
    a6 = Activation('relu')(c6)
    
    #Add spatial Dropout
    if spatial_dropout:
        a6 = SpatialDropout2D(SDRate)(a6)
        
    m3 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(a6)
    
    # Conv Block 4
    c7 = Conv2D(filters=Base*8, 
                     kernel_size=(3,3), strides=(1,1), padding='same')(m3)
    
     #Add batch Normalization
    if batchNormalization:
        c7 = BatchNormalization(axis=-1)(c7)
    
    a7 = Activation('relu')(c7)
    
    #Add spatial Dropout
    if spatial_dropout:
        a7 = SpatialDropout2D(SDRate)(a7)
        
    c8 = Conv2D(filters=Base*8,
                     kernel_size=(3,3), strides=(1,1), padding='same')(a7)
    
     #Add batch Normalization
    if batchNormalization:
        c8 = BatchNormalization(axis=-1)(c8)
    
    a8 = Activation('relu')(c8)
    
    #Add spatial Dropout
    if spatial_dropout:
        a8 = SpatialDropout2D(SDRate)(a8)
        
    m4 = MaxPooling2D(pool_size=(2, 2), strides=(2, 2))(a8)
    
    ##Bottleneck
    # Conv Layer
    c9 = Conv2D(filters=Base*16, 
                     kernel_size=(3,3), strides=(1,1), padding='same')(m4)
    
     #Add batch Normalization
    if batchNormalization:
        c9 = BatchNormalization(axis=-1)(c9)
    
    a9 = Activation('relu')(c9)
    
    #Add spatial Dropout
    if spatial_dropout:
        a9 = SpatialDropout2D(SDRate)(a9)
        
    ##Expansion
    #Conv Block 1
    c10 = Conv2DTranspose(filters=Base*8,
                     kernel_size=(2,2), strides=(2,2), padding='same')(a9)
    c10 = concatenate([a8,c10])
    
    c11 = Conv2D(filters=Base*8,
                     kernel_size=(3,3), strides=(1,1), padding='same')(c10)
    
     #Add batch Normalization
    if batchNormalization:
        c11 = BatchNormalization(axis=-1)(c11)
    
    a10 = Activation('relu')(c11)
    
    #Add spatial Dropout
    if spatial_dropout:
        a10 = SpatialDropout2D(SDRate)(a10)
        
    c12 = Conv2D(filters=Base*8, 
                     kernel_size=(3,3), strides=(1,1), padding='same')(a10)
    
     #Add batch Normalization
    if batchNormalization:
        c12 = BatchNormalization(axis=-1)(c12)
    
    a11 = Activation('relu')(c12)
    
    #Add spatial Dropout
    if spatial_dropout:
        a11 = SpatialDropout2D(SDRate)(a11)
        
    
    #Conv Block 2
    c13 = Conv2DTranspose(filters=Base*4,
                     kernel_size=(2,2), strides=(2,2), padding='same')(a11)
    c13 = concatenate([a6,c13])
    
    c14 = Conv2D(filters=Base*4,
                     kernel_size=(3,3), strides=(1,1), padding='same')(c13)
    
     #Add batch Normalization
    if batchNormalization:
        c14 = BatchNormalization(axis=-1)(c14)
    
    a12 = Activation('relu')(c14)
    
    #Add spatial Dropout
    if spatial_dropout:
        a12 = SpatialDropout2D(SDRate)(a12)
        
    c15 = Conv2D(filters=Base*4, 
                     kernel_size=(3,3), strides=(1,1), padding='same')(a12)
    
     #Add batch Normalization
    if batchNormalization:
        c15 = BatchNormalization(axis=-1)(c15)
    
    a13 = Activation('relu')(c15)
    
    #Add spatial Dropout
    if spatial_dropout:
        a13 = SpatialDropout2D(SDRate)(a13)
        
    
    #Conv Block 3
    c16 = Conv2DTranspose(filters=Base*2,
                     kernel_size=(2,2), strides=(2,2), padding='same')(a13)
    c16 = concatenate([a4,c16])
    
    c17 = Conv2D(filters=Base*2, 
                     kernel_size=(3,3), strides=(1,1), padding='same')(c16)
    
     #Add batch Normalization
    if batchNormalization:
        c17 = BatchNormalization(axis=-1)(c17)
    
    a14 = Activation('relu')(c17)
    
    #Add spatial Dropout
    if spatial_dropout:
        a14 = SpatialDropout2D(SDRate)(a14)
        
    c18 = Conv2D(filters=Base*2,
                     kernel_size=(3,3), strides=(1,1), padding='same')(a14)
    
     #Add batch Normalization
    if batchNormalization:
        c18 = BatchNormalization(axis=-1)(c18)
    
    a15 = Activation('relu')(c18)
    
    #Add spatial Dropout
    if spatial_dropout:
        a15 = SpatialDropout2D(SDRate)(a15)
        
    
    #Conv Block 4
    c19 = Conv2DTranspose(filters=Base,
                     kernel_size=(2,2), strides=(2,2), padding='same')(a15)
    c19 = concatenate([a2,c19])
    
    c20 = Conv2D(filters=Base,
                     kernel_size=(3,3), strides=(1,1), padding='same')(c19)
    
     #Add batch Normalization
    if batchNormalization:
        c20 = BatchNormalization(axis=-1)(c20)
    
    a16 = Activation('relu')(c20)
    
    #Add spatial Dropout
    if spatial_dropout:
        a16 = SpatialDropout2D(SDRate)(a16)
        
    c21 = Conv2D(filters=Base,
                     kernel_size=(3,3), strides=(1,1), padding='same')(a16)
    
     #Add batch Normalization
    if batchNormalization:
        c21 = BatchNormalization(axis=-1)(c21)
    
    a17 = Activation('relu')(c21)
    
    #final layer
    c22 = Conv2D(final_neurons, kernel_size=(3,3), strides=(1,1), padding='same')(a17)
    a18 = Activation(final_afun)(c22)
    
    model = Model(inputs,a18)
    
    model.summary()
    return model

In [6]:
##plotter
import matplotlib.pyplot as plt
def plotter(History):
    #Training vs Validation Learning loss 
    fig_loss = plt.figure(figsize=(4, 4))
    plt.title("Learning curve")
    plt.plot(History.history["loss"], label="loss")
    plt.plot(History.history["val_loss"], label="val_loss")
    plt.plot(np.argmin(History.history["val_loss"]),
             np.min(History.history["val_loss"]),
             marker="x", color="r", label="best model")
    plt.xlabel("Epochs")
    plt.ylabel("Loss Value")
    plt.legend(); 

    
    #Train and test accuracy plot
    fig_dice = plt.figure(figsize=(4,4))
    plt.title("Dice Score Curve")
    plt.plot(History.history["dice_coef"], label="dice_coef")
    plt.plot(History.history["val_dice_coef"], label="val_dice_coef")
    plt.xlabel('Epochs')
    plt.ylabel('Dice Coef')
    plt.legend(); 


    return fig_loss, fig_dice 


In [7]:
import cv2

import SimpleITK as sitk
def create_weight_map(mask, radius, i):
 
    mask = np.uint16(mask)
    # morphology kernel
    kernel = np.ones((radius*2+1,radius*2+1))
    
    # dilate the mask -radius=2
    dilate = cv2.dilate(mask, kernel)
    
    # erode the mask
    erosion = cv2.erode(mask, kernel)
    
    # substract the eroded image from the dilated image
    substraction = dilate-erosion
    
    # save the substraction
   # cv2.imwrite('weight_map_{}.jpg'.format(i),substraction)
    
    #cv2.imshow('image', substraction)
#    mask_dilated = sitk.GrayscaleDilate(mask,radius)
 #   mask_eroded = sitk.GrayscaleErode(mask,radius)
  #  mask_boundary = sitk.Subtract(mask_dilated,mask_eroded)
    
    # save the substraction
   # scipy.misc.imsave('weight_map_{}.jpg'.format(i),mask_boundary)

    return substraction

In [8]:
def weighted_loss(weight_map, weight_strength):
    def weighted_dice_loss(y_true, y_pred):
        y_true_f = K.flatten(y_true)
        y_pred_f = K.flatten(y_pred)
        weight_f = K.flatten(weight_map)
        weight_f = weight_f * weight_strength
        weight_f = 1/(weight_f+1)
        weighted_intersection = K.sum(weight_f * (y_true_f * y_pred_f))
        return -(2. * weighted_intersection + K.epsilon()) / (K.sum(y_true_f) + K.sum(y_pred_f) + K.epsilon())
    return weighted_dice_loss

In [9]:
def combine_generator(gen1, gen2, gen3):
     while True:
        x = gen1.next()
        y = gen2.next()
        w = gen3.next()
        yield([x, w], y)
def generator_with_weights(x_train, y_train, weights_train, batch_size):
    background_value = x_train.min()
    data_gen_args = dict(rotation_range=10., width_shift_range=0.1, height_shift_range=0.1,
    cval=background_value, zoom_range=0.2, horizontal_flip=True)
    image_datagen = ImageDataGenerator(**data_gen_args)
    mask_datagen = ImageDataGenerator(**data_gen_args)
    weights_datagen = ImageDataGenerator(**data_gen_args)
    image_generator = image_datagen.flow(x_train, shuffle=False, batch_size=batch_size,
     seed=1)
    mask_generator = mask_datagen.flow(y_train, shuffle=False, batch_size=batch_size,
     seed=1)
    weight_generator = weights_datagen.flow(weights_train, shuffle=False, batch_size=batch_size,
     seed=1)
    train_generator = combine_generator(image_generator, mask_generator, weight_generator)
    return train_generator

## Task 1

In [ ]:
from tensorflow.keras.optimizers import  Adam
from tensorflow.keras.metrics import Recall, Precision
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import matplotlib.pyplot as plt

#Model parameters
base = 16
image_size = 240
img_ch = 1
batch_size =8
LR = 0.00001
SDRate = 0.5
batch_normalization = True
spatial_dropout = True
metric = 'dice'
epochs = 150
final_neurons= 1 #binary classification
final_afun = "sigmoid" #activation function

#Data loader parameters
p = 0.2
path = '/Lab1/Lab3/MRI/'
fold1 = 'Image'
fold2 = 'Mask'

#Data augmentation parameters
rotation_range = 10
width_shift = 0.1
height_shift_range = 0.1,
rescale = 1./255
horizontal_flip = True

#K-fold cross validation
n_folds = 3
#Load the data

images, masks, Weight_Map = get_train_test_data(fold1, fold2, path, p,image_size, image_size)

#Data augmentation
train_datagen, val_datagen = DataAugmentation(rotation_range,width_shift,height_shift_range,rescale,horizontal_flip)


#Build the model
model = u_net(base,image_size, image_size, img_ch, batch_normalization, SDRate, spatial_dropout,final_neurons, final_afun)
print("model built")
#Compile the model
model.compile(optimizer = Adam(lr=LR), loss = [dice_coef_loss], metrics =[dice_coef, Recall(), Precision()] )
print("compiled!")
 #k-fold crossvalidation loop
cvscores = []
cv = KFold(n_splits=3, random_state=42, shuffle=False)
counter = 1

for train_index, test_index in cv.split(images):
    #train_test split
    print('cross validation fold{}'.format(counter))
    print(train_index)
    x_train, x_val, y_train, y_val = images[train_index], images[test_index], masks[train_index], masks[test_index]

    #Fit the data into the model
   # History = model.fit(x_train, y_train, epochs = epochs, batch_size = batch_size, verbose = 1,
      #                  validation_data = (x_val, y_val))
    History = model.fit_generator(train_datagen.flow(x_train, y_train,batch_size = batch_size), validation_data = val_datagen.flow(x_val, y_val), epochs = epochs, verbose = 1)

   # print("%s: %.2f%%" % (model.metrics_names[1], History.history["val_dice_coef"]))

    fig_loss, fig_dice = plotter(History)
    fig_loss.savefig('Deep-Learning-for-Medical-Application/Lab4/Plots/Task1/Learning_curve_Task{}_fold{}.png'.format(1,counter))
    fig_dice.savefig('Deep-Learning-for-Medical-Application/Lab4/Plots/Task1/Dice_Score_Curve_Task{}_fold{}.png'.format(1,counter))
    cvscores.append(History.history["val_dice_coef"][len(History.history["val_dice_coef"])-1])

    counter=counter+1
    History = None
#print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores)))


/usr/local/lib/python3.5/dist-packages/skimage/io/_io.py:48: UserWarning: `as_grey` has been deprecated in favor of `as_gray`
  warn('`as_grey` has been deprecated in favor of `as_gray`')


Reading: 0/9403  of train images
Readin image /Lab1/Lab3/MRI/Image/Brats17_TCIA_455_1_t1ce_133.png and mask /Lab1/Lab3/MRI/Mask/Brats17_TCIA_455_1_t1ce_133_Tumor.png
Reading: 500/9403  of train images
Readin image /Lab1/Lab3/MRI/Image/Brats17_TCIA_168_1_t1ce_111.png and mask /Lab1/Lab3/MRI/Mask/Brats17_TCIA_168_1_t1ce_111_Tumor.png
Reading: 1000/9403  of train images
Readin image /Lab1/Lab3/MRI/Image/Brats17_TCIA_370_1_t1ce_50.png and mask /Lab1/Lab3/MRI/Mask/Brats17_TCIA_370_1_t1ce_50_Tumor.png
Reading: 1500/9403  of train images
Readin image /Lab1/Lab3/MRI/Image/Brats17_2013_10_1_t1ce_48.png and mask /Lab1/Lab3/MRI/Mask/Brats17_2013_10_1_t1ce_48_Tumor.png
Reading: 2000/9403  of train images
Readin image /Lab1/Lab3/MRI/Image/Brats17_TCIA_372_1_t1ce_73.png and mask /Lab1/Lab3/MRI/Mask/Brats17_TCIA_372_1_t1ce_73_Tumor.png
Reading: 2500/9403  of train images
Readin image /Lab1/Lab3/MRI/Image/Brats17_TCIA_150_1_t1ce_57.png and mask /Lab1/Lab3/MRI/Mask/Brats17_TCIA_150_1_t1ce_57_Tumor.png


## Task 2

In [1]:
# Main code

from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.metrics import Recall, Precision
from sklearn.model_selection import StratifiedKFold

#Model parameters
base = 16
image_size = 240
img_ch = 1
batch_size =8
LR = 0.00001
SDRate = 0.5
batch_normalization = True
spatial_dropout = True
metric = 'dice'
epochs = 150
final_neurons= 1 #binary classification
final_afun = "sigmoid" #activation function

#Data loader parameters
p = 0.2
path = '/Lab1/Lab3/MRI/'
fold1 = 'Image'
fold2 = 'Mask'

#Data augmentation parameters
data_augmentation = True
rotation_range = 10
width_shift = 0.1
height_shift_range = 0.1,
rescale = 1./255
horizontal_flip = True

#K-fold cross validation
n_folds = 3
weight_strength = 1
#Load the data
images, masks, weight_maps = get_train_test_data(fold1, fold2, path, p,image_size, image_size)

#Data augmentation
train_datagen, val_datagen = DataAugmentation(rotation_range,width_shift,height_shift_range,rescale,horizontal_flip)


#Build the model
model = u_net(base,image_size, image_size, img_ch, batch_normalization, SDRate, spatial_dropout,final_neurons, final_afun)

#k-fold crossvalidation loop
for _ in range(n_folds):
    #train_test split
    x_train,x_val,y_train,y_val = train_test_split(images,masks,test_size = p, shuffle = False)
    weight_train = weight_map[0:x_train.shape[0]-1][:,:]
    weight_val = weight_map[x_train.shape[0]:][:,:]
    #Compile the model
    model.compile(optimizer = Adam(lr=LR), loss = weighted_loss(weight_maps, weight_strength), metrics =[dice_coef, Recall(), Precision()] )
    if data_augmentation:
        #Fit the data into the model
        train_generator = generator_with_weights(x_train, y_train, weight_train, Batch_size)
        History = model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=epochs, 
                                            verbose=1, max_queue_size=1, validation_steps=len(x_val),
                                            validation_data=([x_val, weight_val], y_val), shuffle=True, class_weight='auto')
    else:
        History = model.fit([x_train, weight_train], y_train, epochs = epochs, batch_size = batch_size, verbose = 1,
                        validation_data = ([x_val, weight_val], y_val))
        
    
    print("%s: %.2f%%" % (model.metrics_names[1], History.history["val_dice_coef"]*100))
    cvscores.append(History.history["val_dice_coef"][len(History.history["val_dice_coef"])-1] * 100)
    plotter(History)
print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores)))


NameError: name 'get_train_test_data' is not defined

## Task 3

In [ ]:

T = 2 #number of cycles

x_train,x_val,y_train,y_val = train_test_split(images,masks,test_size = p, shuffle = False)

for s in range(T):
    #train_test split
    
    weight_train = weight_map[0:x_train.shape[0]-1][:,:]
    weight_val = weight_map[x_train.shape[0]:][:,:]
    #Compile the model
    model.compile(optimizer = Adam(lr=LR), loss = weighted_loss(weight_maps, weight_strength), metrics =[dice_coef, Recall(), Precision()] )
    if data_augmentation:
        #Fit the data into the model
        train_generator = generator_with_weights(x_train, y_train, weight_train, Batch_size)
        History = model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=epochs, 
                                            verbose=1, max_queue_size=1, validation_steps=len(x_val),
                                            validation_data=([x_val, weight_val], y_val), shuffle=True, class_weight='auto')
    else:
        History = model.fit([x_train, weight_train], y_train, epochs = epochs, batch_size = batch_size, verbose = 1,
                        validation_data = ([x_val, weight_val], y_val))
        
    
    val_predictions=model.predict(x_val,batch_size=int(batch_size/2))
    train_predictions=model.predict(x_train,batch_size=int(batch_size/2))
    
    model_predictions[(s*1):((s+1)*1)] = val_predictions
    
    #Ask, we need to change the channels to 2

    x_train,x_val,y_train,y_val = train_test_split(images,masks,test_size = p, shuffle = False)
    
    x_train=np.concatenate((x_train,train_predictions),axis=-1)
    
    x_val=np.concatenate((x_val,val_predictions),axis=-1)
        
    
    print("%s: %.2f%%" % (model.metrics_names[1], History.history["val_dice_coef"]*100))
    cvscores.append(History.history["val_dice_coef"][len(History.history["val_dice_coef"])-1] * 100)
    plotter(History)
print("%.2f%% (+/- %.2f%%)" % (numpy.mean(cvscores), numpy.std(cvscores)))    
